[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb)

In [1]:
!wget https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.tar.gz

--2018-06-19 05:32:15--  https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.tar.gz
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz [following]
--2018-06-19 05:32:15--  https://www.kaggle.com/account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 8229 (8.0K) [text/html]
Saving to: ‘fer2013.tar.gz’

fer2013.tar.gz      100%[===================>]   8.04K  --.-KB/s    in 0s      

2018-06-19 05:32:15 (149 MB/s) - ‘fer2013.tar.gz’ saved [8229/8229]



In [1]:
from google.colab import files
uploaded = files.upload()

Saving fer2013.tar to fer2013.tar


In [3]:
!ls

datalab  fer2013.tar  fer2013.tar.gz


In [42]:
import os
!export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ.pop('TF_ENABLE_WINOGRAD_NONFUSED', None)

'1'

In [2]:
!tar xvf fer2013.tar
!ls

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/
datalab  fer2013  fer2013.tar


In [0]:

import csv
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, InputLayer
from keras.layers import Convolution2D, SeparableConv2D, MaxPooling2D, BatchNormalization 
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

In [0]:
img_rows, img_cols = 48, 48
batch_size = 128
classes = 7
epoch = 100
img_channels = 1

In [0]:
import csv
f = open('fer2013/fer2013.csv')
csv_f = csv.reader(f)

train_x = []
train_y = []
val_x =[]
val_y =[]


In [9]:
print(csv_f)

In [0]:
num=0
for row in csv_f:
  #print(row)
  num = num +1
  #print(num)
  temp_list = []
  for pixel in row[1].split( ):
    temp_list.append(int(pixel))

  if str(row[2]) == "Training":
    train_y.append(int(row[0]))
    train_x.append(temp_list) 
  elif str(row[2]) == "PublicTest":
    val_y.append(int(row[0]))
    val_x.append(temp_list)

In [0]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

In [0]:
train_x = train_x.reshape(train_x.shape[0], 48, 48)
train_x = train_x.reshape(train_x.shape[0], 1, 48, 48)
train_y = np_utils.to_categorical(train_y, 7)

In [0]:
val_x = val_x.reshape(val_x.shape[0], 48, 48)
val_x = val_x.reshape(val_x.shape[0], 1, 48, 48)
val_y = np_utils.to_categorical(val_y, 7)

In [0]:
train_x = train_x.astype('float32')
train_x = train_x / 255.0
val_x = val_x.astype('float32')
val_x = val_x / 255.0


In [0]:
input_shape = (img_channels, img_rows, img_cols)
model = Sequential()
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same',
                            name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(PReLU())
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=1024, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(SeparableConv2D(filters=1024, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(PReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(Flatten())

model.add(Dense(2048))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Dropout(.5))

model.add(Dense(2048))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Dropout(.5))

#model.add(InputLayer(input_shape=(1, 2, 2014)))
#model.add(GlobalAveragePooling2D())
model.add(Dense(7))
model.add(Activation('softmax'))

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (SeparableConv2D (None, 1, 48, 64)         3568      
_________________________________________________________________
batch_normalization_207 (Bat (None, 1, 48, 64)         256       
_________________________________________________________________
p_re_lu_180 (PReLU)          (None, 1, 48, 64)         3072      
_________________________________________________________________
separable_conv2d_146 (Separa (None, 1, 48, 64)         4736      
_________________________________________________________________
batch_normalization_208 (Bat (None, 1, 48, 64)         256       
_________________________________________________________________
p_re_lu_181 (PReLU)          (None, 1, 48, 64)         3072      
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 1, 24, 64)         0         
__________

In [0]:
model.compile(optimizer='nadam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
filepath='Model.{epoch:02d}.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')


In [0]:
model.load_weights('Model.49-0.4960.hdf5')

In [73]:
# PReLU
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=50, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 34s 1ms/step - loss: 2.0557 - acc: 0.2439 - val_loss: 12.0170 - val_acc: 0.2497

Epoch 00001: saving model to Model.01.hdf5
Epoch 2/50
26368/28709 [==========================>...] - ETA: 1s - loss: 2.0282 - acc: 0.2418

28709/28709 [==============================] - 21s 733us/step - loss: 2.0266 - acc: 0.2429 - val_loss: 10.3507 - val_acc: 0.2488

Epoch 00002: saving model to Model.02.hdf5
Epoch 3/50
28709/28709 [==============================] - 21s 726us/step - loss: 1.9263 - acc: 0.2712 - val_loss: 2.1679 - val_acc: 0.2586

Epoch 00003: saving model to Model.03.hdf5
Epoch 4/50
10624/28709 [==========>...................] - ETA: 12s - loss: 1.9367 - acc: 0.2775

28709/28709 [==============================] - 21s 726us/step - loss: 1.8957 - acc: 0.2869 - val_loss: 1.9084 - val_acc: 0.1950

Epoch 00004: saving model to Model.04.hdf5
Epoch 5/50
28709/28709 [==============================] - 21s 732us/step - loss: 1.9289 - acc: 0.2989 - val_loss: 2.0679 - val_acc: 0.2926

Epoch 00005: saving model to Model.05.hdf5
Epoch 6/50
 7808/28709 [=======>......................] - ETA: 15s - loss: 1.8489 - acc: 0.3051

28709/28709 [==============================] - 21s 736us/step - loss: 1.8734 - acc: 0.3097 - val_loss: 1.9956 - val_acc: 0.3031

Epoch 00006: saving model to Model.06.hdf5
Epoch 7/50
28709/28709 [==============================] - 21s 734us/step - loss: 1.8495 - acc: 0.3175 - val_loss: 2.0653 - val_acc: 0.3380

Epoch 00007: saving model to Model.07.hdf5
Epoch 8/50
 7296/28709 [======>.......................] - ETA: 15s - loss: 1.8555 - acc: 0.3159

28709/28709 [==============================] - 21s 735us/step - loss: 1.8655 - acc: 0.3051 - val_loss: 1.7540 - val_acc: 0.3149

Epoch 00008: saving model to Model.08.hdf5
Epoch 9/50
28709/28709 [==============================] - 21s 729us/step - loss: 1.8347 - acc: 0.3156 - val_loss: 1.9117 - val_acc: 0.3544

Epoch 00009: saving model to Model.09.hdf5
Epoch 10/50
 7168/28709 [======>.......................] - ETA: 15s - loss: 1.8249 - acc: 0.3117

...

28709/28709 [==============================] - 21s 732us/step - loss: 1.5206 - acc: 0.4226 - val_loss: 1.4719 - val_acc: 0.4425

Epoch 00042: saving model to Model.42.hdf5
Epoch 43/50
28709/28709 [==============================] - 21s 731us/step - loss: 1.5063 - acc: 0.4290 - val_loss: 1.5562 - val_acc: 0.4099

Epoch 00043: saving model to Model.43.hdf5
Epoch 44/50
 6912/28709 [======>.......................] - ETA: 15s - loss: 1.4717 - acc: 0.4361

28709/28709 [==============================] - 21s 733us/step - loss: 1.4788 - acc: 0.4384 - val_loss: 1.4797 - val_acc: 0.4369

Epoch 00044: saving model to Model.44.hdf5
Epoch 45/50
28709/28709 [==============================] - 21s 732us/step - loss: 1.4928 - acc: 0.4331 - val_loss: 1.7750 - val_acc: 0.4230

Epoch 00045: saving model to Model.45.hdf5
Epoch 46/50
 6912/28709 [======>.......................] - ETA: 15s - loss: 1.5120 - acc: 0.4333

28709/28709 [==============================] - 21s 734us/step - loss: 1.5064 - acc: 0.4353 - val_loss: 1.6163 - val_acc: 0.3990

Epoch 00046: saving model to Model.46.hdf5
Epoch 47/50
28709/28709 [==============================] - 21s 736us/step - loss: 1.5124 - acc: 0.4315 - val_loss: 1.4717 - val_acc: 0.4369

Epoch 00047: saving model to Model.47.hdf5
Epoch 48/50
 6912/28709 [======>.......................] - ETA: 15s - loss: 1.4713 - acc: 0.4439

28709/28709 [==============================] - 21s 732us/step - loss: 1.4698 - acc: 0.4415 - val_loss: 1.4558 - val_acc: 0.4347

Epoch 00048: saving model to Model.48.hdf5
Epoch 49/50
28709/28709 [==============================] - 21s 733us/step - loss: 1.4594 - acc: 0.4448 - val_loss: 1.4554 - val_acc: 0.4433

Epoch 00049: saving model to Model.49.hdf5
Epoch 50/50
 6912/28709 [======>.......................] - ETA: 15s - loss: 1.4271 - acc: 0.4528

28709/28709 [==============================] - 21s 729us/step - loss: 1.4260 - acc: 0.4547 - val_loss: 1.4508 - val_acc: 0.4466

Epoch 00050: saving model to Model.50.hdf5
--- 1141.6047801971436 seconds ---


In [50]:
# input normalized
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 162s 6ms/step - loss: 2.0189 - acc: 0.2407 - val_loss: 1.8390 - val_acc: 0.2856
Epoch 2/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.8329 - acc: 0.2909 - val_loss: 1.7546 - val_acc: 0.3391
Epoch 3/10
  896/28709 [..............................] - ETA: 2:27 - loss: 1.7662 - acc: 0.3058

28709/28709 [==============================] - 157s 5ms/step - loss: 1.7695 - acc: 0.3230 - val_loss: 1.8986 - val_acc: 0.3093
Epoch 4/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.7122 - acc: 0.3484 - val_loss: 1.6657 - val_acc: 0.3611
Epoch 5/10
10880/28709 [==========>...................] - ETA: 1:34 - loss: 1.6901 - acc: 0.3509

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6773 - acc: 0.3604 - val_loss: 1.7014 - val_acc: 0.3494
Epoch 6/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.6553 - acc: 0.3702 - val_loss: 1.6194 - val_acc: 0.3859
Epoch 7/10
13312/28709 [============>.................] - ETA: 1:20 - loss: 1.6141 - acc: 0.3857

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6129 - acc: 0.3857 - val_loss: 1.6001 - val_acc: 0.3923
Epoch 8/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.5920 - acc: 0.3924 - val_loss: 1.6594 - val_acc: 0.4035
Epoch 9/10
13824/28709 [=============>................] - ETA: 1:18 - loss: 1.5641 - acc: 0.4065

28709/28709 [==============================] - 157s 5ms/step - loss: 1.5559 - acc: 0.4044 - val_loss: 1.5507 - val_acc: 0.4113
Epoch 10/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.5341 - acc: 0.4100 - val_loss: 1.5732 - val_acc: 0.4068
--- 1574.3259291648865 seconds ---


In [0]:

import time
start_time = time.time()

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(train_x)

model.fit_generator(datagen.flow(train_x, train_y,
                    batch_size=batch_size),
                    steps_per_epoch=(train_x.shape[0]/batch_size),
                    epochs=10,
                    validation_data=(val_x, val_y),
                    callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))